In [1]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(15):
    # Directory containing your event files
    log_dir = f"./dfl_15_fully_connected/participant_{i}"
    date_str = "02/07/2024 10:22:20" # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step     value                     timestamp             tag
0  1294  5.000000 2024-07-02 10:27:40.790342912           Round
1   647  0.123888 2024-07-02 10:27:42.863721984       Test/Loss
2   647  0.915617 2024-07-02 10:27:42.863782144   Test/Accuracy
3   647  0.916635 2024-07-02 10:27:42.863806464  Test/Precision
4   647  0.915617 2024-07-02 10:27:42.863826688     Test/Recall
2024-07-02 10:29:53 2024-07-01 11:54:03
                step     value                     timestamp
tag                                                         
Test/Accuracy   1197  0.928853 2024-07-02 10:29:51.436961280
Test/F1Score    1197  0.919388 2024-07-02 10:29:51.437045248
Test/Loss       1197  0.083062 2024-07-02 10:29:51.436893440
Test/Precision  1197  0.921726 2024-07-02 10:29:51.436995328
Test/Recall     1197  0.928853 2024-07-02 10:29:51.437019904
   step         value                     timestamp                     tag
0   204  1.000000e+02 2024-07-02 10:25:51.480782848   Resources/CPU_perce

In [2]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,1197,0.928853,2024-07-02 10:29:51.436961280,participant_0
Test/F1Score,1197,0.919388,2024-07-02 10:29:51.437045248,participant_0
Test/Loss,1197,0.083062,2024-07-02 10:29:51.436893440,participant_0
Test/Precision,1197,0.921726,2024-07-02 10:29:51.436995328,participant_0
Test/Recall,1197,0.928853,2024-07-02 10:29:51.437019904,participant_0
...,...,...,...,...
Test/Accuracy,1623,0.952283,2024-07-02 10:29:51.583958784,participant_14
Test/F1Score,1623,0.952574,2024-07-02 10:29:51.584044800,participant_14
Test/Loss,1623,0.060566,2024-07-02 10:29:51.583902464,participant_14


In [3]:
concat_df.groupby("tag").mean()

/tmp/ipykernel_37324/1972488931.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean()


,step,value
tag,,
Test/Accuracy,1546.0,0.938465
Test/F1Score,1546.0,0.934082
Test/Loss,1546.0,0.092111
Test/Precision,1546.0,0.938659
Test/Recall,1546.0,0.938465


In [5]:
concat_df.to_csv("results_csvs/dfl_15_fully_connected.csv")